This notebook is used to get residence-time distribution (RTD) for the entire aquifer from an existing MODFLOW model. It is possible to read in any group or label from a 3D array and make RTDs for those groups. The approach is to 
* read an existing model
* create flux-weighted particle starting locations in every cell
* run MODPATH and read endpoints
* fit parametric distributions

This notebook fits parametric distributions. Another notebook creates flux-weighted particles.

In [ ]:
%matplotlib notebook
__author__ = 'Jeff Starn'

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
from IPython.display import Image
from IPython.display import Math
from ipywidgets import interact, Dropdown
from IPython.display import display

import os
import sys
import shutil
import pickle
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.ticker as mt
import matplotlib.patches as patches

import flopy as fp
import imeth
import fit_parametric_distributions
import pandas as pd
import gdal
import scipy.stats as ss
import scipy.optimize as so
from scipy.interpolate import Rbf
from scipy.interpolate import griddata


# Preliminary stuff

## Set user-defined variables

MODFLOW and MODPATH use elapsed time and are not aware of calendar time. To place MODFLOW/MODPATH elapsed time on the calendar, two calendar dates were specified at the top of the notebook: the beginning of the first stress period (`mf_start_date`) and when particles are to be released (`mp_release_date`). The latter date could be used in many ways, for example to represent a sampling date, or it could be looped over to create a time-lapse set of ages. 

## Loop through home directory to get list of name files

In [ ]:
homes = ['../Models']
fig_dir = '../Figures'

if not os.path.exists(fig_dir):
    os.mkdir(dst)

mfpth = '../executables/MODFLOW-NWT_1.0.9/bin/MODFLOW-NWT_64.exe'
mp_exe_name = '../executables/modpath.6_0/bin/mp6.exe' 

mf_start_date_str = '01/01/1900' 
mp_release_date_str = '01/01/2020' 

logtransform = False

weight_scheme = 'flow'
# weight_scheme = 'volume'

# dist_list = [ss.invgauss, ss.gamma, ss.weibull_min]
dist_list = [ss.weibull_min]

por = 0.20

dir_list = []
mod_list = []
i = 0

for home in homes:
    if os.path.exists(home):
        for dirpath, dirnames, filenames in os.walk(home):
            for f in filenames:
                if os.path.splitext(f)[-1] == '.nam':
                    mod = os.path.splitext(f)[0]
                    mod_list.append(mod)
                    dir_list.append(dirpath)
                    i += 1
print('    {} models read'.format(i))

model_area = Dropdown(
    options=mod_list,
    description='Model:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(model_area)


In [ ]:
model = model_area.value
model_ws = [item for item in dir_list if model in item][0]
nam_file = '{}.nam'.format(model)
print("working model is {}".format(model_ws))

##  Create names and path for model workspace. 

The procedures in this notebook can be run from the notebook or from a batch file by downloading the notebook as a Python script and uncommenting the following code and commenting out the following block. The remainder of the script has to be indented to be included in the loop.  This may require familiarity with Python. 

In [ ]:
# for pth in dir_list:
#     model = os.path.normpath(pth).split(os.sep)[2]
#     model_ws = [item for item in dir_list if model in item][0]
#     nam_file = '{}.nam'.format(model)
#     print("working model is {}".format(model_ws))

# Load an existing model

In [ ]:
print ('Reading model information')

fpmg = fp.modflow.Modflow.load(nam_file, model_ws=model_ws, exe_name=mfpth, version='mfnwt', 
                               load_only=['DIS', 'BAS6', 'UPW', 'OC'], check=False)

dis = fpmg.get_package('DIS')
bas = fpmg.get_package('BAS6')
upw = fpmg.get_package('UPW')
oc = fpmg.get_package('OC')

delr = dis.delr
delc = dis.delc
nlay = dis.nlay
nrow = dis.nrow
ncol = dis.ncol
bot = dis.getbotm()
top = dis.gettop()

hnoflo = bas.hnoflo
ibound = np.asarray(bas.ibound.get_value())
hdry = upw.hdry

print ('   ... done') 

## Specification of time in MODFLOW/MODPATH

There are several time-related concepts used in MODPATH.
* `simulation time` is the elapsed time in model time units from the beginning of the first stress period
* `reference time` is an arbitrary value of `simulation time` that is between the beginning and ending of `simulation time`
* `tracking time` is the elapsed time relative to `reference time`. It is always positive regardless of whether particles are tracked forward or backward
* `release time` is when a particle is released and is specified in `tracking time`

In [ ]:
# setup dictionaries of the MODFLOW units for proper labeling of figures.
lenunit = {0:'undefined units', 1:'feet', 2:'meters', 3:'centimeters'}
timeunit = {0:'undefined', 1:'second', 2:'minute', 3:'hour', 4:'day', 5:'year'}

# Create dictionary of multipliers for converting model time units to days
time_dict = dict()
time_dict[0] = 1.0 # undefined assumes days, so enter conversion to days
time_dict[1] = 24 * 60 * 60
time_dict[2] = 24 * 60
time_dict[3] = 24
time_dict[4] = 1.0
time_dict[5] = 1.0

In [ ]:
# convert string representation of dates into Python datetime objects
mf_start_date = dt.datetime.strptime(mf_start_date_str , '%m/%d/%Y')
mp_release_date = dt.datetime.strptime(mp_release_date_str , '%m/%d/%Y')

# convert simulation time to days from the units specified in the MODFLOW DIS file
sim_time = np.append(0, dis.get_totim())
sim_time /= time_dict[dis.itmuni]

# make a list of simulation time formatted as calendar dates
date_list = [mf_start_date + dt.timedelta(days = item) for item in sim_time]

# reference time and date are set to the end of the last stress period
ref_time = sim_time[-1]
ref_date = date_list[-1]

# release time is calculated in tracking time (for particle release) and 
# in simulation time (for identifying head and budget components)
release_time_trk = np.abs((ref_date - mp_release_date).days)
release_time_sim = (mp_release_date - mf_start_date).days

In [ ]:
src = os.path.join(model_ws, fpmg.namefile)
name_file_df = pd.read_table(src, header=None, comment='#', delim_whitespace=True, 
              names=['package', 'unit', 'filename', 'type'])

name_file_df['package'] = name_file_df.package.str.lower()
name_file_df.set_index('unit', inplace=True)

head_file_name = name_file_df.loc[oc.iuhead, 'filename']
bud_file_name = name_file_df.loc[oc.get_budgetunit(), 'filename']

src = os.path.join(model_ws, bud_file_name)
bud_obj = fp.utils.CellBudgetFile(src)
all_bud_df = pd.DataFrame(bud_obj.recordarray)

# convert to zero base
all_bud_df['kper'] = all_bud_df['kper'] - 1
all_bud_df['kstp'] = all_bud_df['kstp'] - 1

# add calendar date (not used at this time)
all_bud_df['date'] = mf_start_date + pd.to_timedelta(all_bud_df.totim, unit='days')

# group by period and step
kdf = all_bud_df.groupby(['kper', 'kstp']).median()

# find the latest group index that includes the release date
idx = kdf.loc[(kdf.totim >= release_time_sim).idxmax(), :].name

# switch period and step 
kstpkper = tuple(item for item in idx[-1::-1])

# extract the budget records for the specified period and step
bud_df = all_bud_df.query('kstp=={} and kper=={}'.format(*kstpkper)).copy()

bud_df.loc[:, 'per_num'] = bud_df.totim.factorize()[0]
num_rec = bud_df.shape[0]

src = os.path.join(model_ws, head_file_name)
hd_obj = fp.utils.HeadFile(src)
head_df = pd.DataFrame(hd_obj.recordarray)

heads = hd_obj.get_data(kstpkper=kstpkper)

heads[np.isclose(hnoflo, heads)] = np.nan
heads[np.isclose(hdry, heads)] = np.nan
hin = np.argmax(np.isfinite(heads), axis=0)
row, col = np.indices((hin.shape))
water_table = heads[hin, row, col]

# Process endpoint information

## Read endpoint file

## Review zones and create zone groups for processing

# Fit parametric distributions

## Subsample endpoints
This next cell takes `s` number of stratified random samples from the endpoints. This is in order to make the curve fitting much faster. 50,000 samples seems to work pretty well.

## Calculate summary statistics

Two definitions of young fraction are included. The first is relative to a particle travel time (age) cut-off in years. For steady-state models, it is independent of time and describes the young fraction of the RTD. For transient models, the young fraction depends on the particle release time. The second definition is relative to a calendar date; only particles that recharged after that date are included in the summary statistics. It is equal to the first definition by assuming particle release in 2017. It may be useful for assessing the breakthrough of chemicals released at a known date. For both definitions, the young fraction is described by the number of particles, the mean travel time of those particles, and the fraction of total particles.



In [ ]:
src = os.path.join(model_ws, 'zone_df.csv')
zone_df = pd.read_csv(src, index_col=0)

# Plot CDF and PDFS

## Plot CDFs for each model

In [ ]:
dst = os.path.join(model_ws, 'RTD')
if not os.path.exists(dst):
    os.mkdir(dst)

In [ ]:
src = os.path.join(model_ws, 'tau.csv')
tau_table = pd.read_csv(src, index_col=0)

In [ ]:
dst = os.path.join(model_ws, 'fit_dict_{}.pickle'.format(weight_scheme))
with open(dst, 'rb') as f:
        fit_dict = pickle.load(f)

In [ ]:
for group in fit_dict.keys():
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), sharey=False)

    dist = dist_list[0]
    uname = 'uni_{}'.format(dist.name)
    aname = 'add_{}'.format(dist.name)

    part_tt = fit_dict[group]['tt']['rt'] 
    part_cdf = fit_dict[group]['tt']['rt_cdf']
    
    if logtransform:
        loglabel = 'log'
    else:
        loglabel = 'linear'
    
    ax.plot(part_tt, part_cdf, label='Particle', lw=5, color='r', alpha=0.4)

    try:
        uni_cdf = fit_dict[group]['cdf'][uname]
        ax.plot(part_tt, uni_cdf, label='One component', color='k', lw=1)
    except Exception as e: 
        print(e)

    fy = -9999
    try:
        ep_exp = fit_dict[group]['par'][aname]
        add_cdf = fit_dict[group]['cdf'][aname]
        fy = ep_exp[6]
        ax.plot(part_tt, add_cdf, label='Explicitly mixed', color='r', lw=1)
    except Exception as e: 
        print(e)

    first = part_tt.min()
    try:
        tau = tau_table.loc[group, 'tau']
        expon = ss.expon(first, tau)
        exy = expon.cdf(part_tt)
        ax.plot(part_tt, exy, label='Exponential', color='b', lw=1, ls='dashed')
    except Exception as e: 
        print(e)

    ax.set_xscale('log')
    ax.set_xlim(first, 100000)
    ax.set_ylim(0, 1)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + 0.15, box.width * 1.10, box.height * 0.8])
    ax.set_ylabel('Cumulative frequency', fontsize=8)
    ax.set_xlabel('Residence time ({}), in years'.format(loglabel), fontsize=8)
    ax.legend(loc=0, frameon=False, fontsize=8, handlelength=3, numpoints=1, ncol=2, 
              bbox_to_anchor=(1.0, -0.2))
    ax.set_title('{2:} {3:}\n{0:} with {1:0.0f}% RTD$_1$'.format(dist.name, fy*100, fpmg.name, group), fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=8)

    dst = 'Cumulative RTD ({}) for all distributions in zone {}--{}.png'.format(loglabel, group, model)
    dst_pth = os.path.join(model_ws, 'RTD', dst)
    plt.savefig(dst_pth, dpi=300)
#     plt.close()

## Plot PDF for explicit RTD mixtures

Set the number of particles at which to compute particle density `numparts`
and x axis maxmimum `axmax` for the best looking plot.

In [ ]:
numparts = 1000
axmax = 1000

In [ ]:
for group in fit_dict.keys():
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), sharey=False)

    dist = dist_list[0]
    uname = 'uni_{}'.format(dist.name)
    aname = 'add_{}'.format(dist.name)

    part_cdf = fit_dict[group]['tt']['rt_cdf']
    if logtransform:
        loglabel = 'log'
        part_tt = np.exp(fit_dict[group]['tt']['rt'] )

    else:
        loglabel = 'linear'
        part_tt = fit_dict[group]['tt']['rt'] 
        
    # trim the ages to get a better plot of the pdf
    # eliminate ages < 1 year and > 1,000,000 years
    lo_trim = part_tt > 1
    hi_trim = part_tt < 1.E+06
    trim = np.logical_and(lo_trim, hi_trim)
    
    part_tt = part_tt[trim]
    part_cdf = part_cdf[trim]
    
    x = np.linspace(part_tt.min(), part_tt.max(), numparts)
    yi = np.interp(x, part_tt, part_cdf)
    pdf = np.diff(yi) / np.diff(x)
    xav = (x[0:-1] + x[1:]) / 2
    
    lx = np.logspace(np.log10(part_tt.min()), np.log10(part_tt.max()), numparts)
    lyi = np.interp(lx, part_tt, part_cdf)
    pdf = np.diff(lyi) / np.diff(lx)
    xav = (lx[0:-1] + lx[1:]) / 2
    
    ax.plot(xav, pdf, linestyle='None', marker='.', mfc='0.20', mew=0, 
            label='Particle RTD', ms=5, alpha=0.9)

    fy = 0
    
    try:
        ep_exp = fit_dict[group]['par'][aname]
        pdf_e_exp = dist(*ep_exp[0:3]).pdf(xav) 
        ax.plot(xav, pdf_e_exp, color='blue', linestyle='dashed', linewidth=1, label='RTD$_1$')
        mean_age_early = dist(*ep_exp[0:3]).mean()
        ax.axvline(mean_age_early, 0, 0.1, color='blue', label='RTD$_1$ mean age', lw=2)

        ep_exp = fit_dict[group]['par'][aname]
        pdf_l_exp = dist(*ep_exp[3:6]).pdf(xav) 
        ax.plot(xav, pdf_l_exp, color='green', linestyle='dashed', linewidth=1, label='RTD$_2$')
        mean_age_late = dist(*ep_exp[3:6]).mean()
        ax.axvline(mean_age_late, 0, 0.1, color='green', label='RTD$_2$ mean age', lw=2)

        fy = ep_exp[6]
        combined_exp = fy * pdf_e_exp + (1 - fy) * pdf_l_exp
        ax.fill_betweenx(combined_exp, xav, color='r', linewidth=1, label='Composite RTD', alpha=0.3)
        mean_age_mixed = (xav[1:] * combined_exp[1:] * np.diff(xav)).sum()
        ax.axvline(mean_age_mixed, 0, 0.1, color='red', label='Composite mean age', lw=2)

    except Exception as e: 
        print(e)

    ax.set_xscale('log')
    ax.set_xlim(xav.min(), axmax)
    ax.set_ylim(0, )
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + 0.15, box.width * 1.10, box.height * 0.8])
    ax.set_ylabel('Density in 1 / years', fontsize=8)
    ax.set_xlabel('Residence time ({}), in years'.format(loglabel), fontsize=8)
    ax.legend(loc=0, frameon=False, fontsize=8, handlelength=3, numpoints=1, ncol=3, 
              bbox_to_anchor=(1.1, -0.15))
    ax.set_title('{2:} {3:}\n{0:} with {1:0.0f}% RTD$_1$'.format(dist.name, fy*100, fpmg.name, group), fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=8)

    dst = 'Density of RTD ({}) for all distributions in zone {}--{}.png'.format(loglabel, group, model)
    dst_pth = os.path.join(model_ws, 'RTD', dst)
    plt.savefig(dst_pth, dpi=300)

    # plt.close()    